<a href="https://colab.research.google.com/github/Zul-Qarnain/Math6_project/blob/main/MATH6_PROJECT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# New section